In [1]:
import numpy as np 
import pandas as pd
import re 
import os

In [2]:
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']

## ---------   Handling the data from 2019 - 2021  -------------

In [4]:
df2 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\2019-2021\Data19-21.xlsx")

In [5]:
df2.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Country,Product Name,Company
0,Import,2021,1,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,NaN,EMAMECTIN TECH 95%,KRISHI RASAYAN
1,Import,2021,1,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,NaN,EMAMECTIN TECH 95%,WILLOWOOD CHEMICALS PRIVATE LIMITED
2,Import,2021,2,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,NaN,EMAMECTIN Others,SYNGENTA INDIA LIMITED
3,Import,2021,2,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,NaN,EMAMECTIN Others,SYNGENTA INDIA LIMITED
4,Import,2021,3,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,NaN,EMAMECTIN TECH 95%,CRYSTAL CROP PROTECTION LIMITED


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24595 entries, 0 to 24594
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Type                        24595 non-null  object        
 1   Year                        24595 non-null  int64         
 2   Month                       24595 non-null  int64         
 3   Date                        24595 non-null  datetime64[ns]
 4   IMPORTER/EXPORTER NAME      24595 non-null  object        
 5   ORIGIN/DESTINATION COUNTRY  24595 non-null  object        
 6   PRODUCT DESCRIPTION         24595 non-null  object        
 7   QTY                         24595 non-null  float64       
 8   UNIT                        24595 non-null  object        
 9   UNIT PRICE (USD)            24595 non-null  float64       
 10  Total_VALUE_USD             24595 non-null  float64       
 11  INVOICE CURRENCY            24594 non-null  object    

In [7]:
df2.drop(['Molecule Name','Product Name'],axis =1 ,inplace = True)

### Looping through the Product Description column and searching for elements in the Product column and the "2 4 d " AI.

In [8]:
map_24d = { v: '2 4 d' for v in ['2 4-d', '24 d', '2,4-d', '2 4 d', '2 4- d', '2 4 -d', '2 4d',
       '2, 4 d', '2, 4-d', '2,4 - d', '2,4 d', '2,4- d', '2,4d', '2-4-d',
       '24d', '2-4 d', '24- d', '2-4- d', '2-4d', '24-d', '2, 4- d',
       '2, 4d', '2- 4-d']}


def find_prod(x):
   
    #a = ('azoxystrobin')
    if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample',x) :
        return ""
    temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
    if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
        temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
    #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
        #temp_products += a
    if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
        if len(temp_products)<1:
            temp_products += 'fertilizer'
    if len(temp_products)>1 :
        temp_products = [i for i in temp_products if i!='pesticide']
        temp_products = [i for i in temp_products if i!='pesticides']
        temp_products = [i for i in temp_products if i!='rynaxypyr']
    return temp_products

  

In [9]:
df2['PRODUCT DESCRIPTION'] = df2['PRODUCT DESCRIPTION'].map(str)

In [10]:
temp = df2['PRODUCT DESCRIPTION'].apply(lambda x: find_prod(x))
# temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")

In [20]:
df2.loc[np.logical_and(df2['UNIT'].gt('GMS'), np.less_equal(df2['QTY'], 1)), 'Product'] = ''
#if df2.loc[df2['Quantity']]== 0 :
#   df2['Product'] = ''
df2.loc[df2["QTY"]==0, "Product"] = ""

In [21]:
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('mancozebe','mancozeb',inplace = True)
temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
temp_processed.replace('glyphosatee','glyphosate',inplace = True)
temp_processed.replace('','Others',inplace = True)

In [22]:
df2['Product'] = temp_processed

In [23]:
df2['Product'] = df2['Product'].fillna('Others')

In [24]:
df2['Product'].value_counts()

mancozeb                                 10955
glufosinate                               2132
glyphosate                                1413
pendimethalin                             1006
metribuzin                                 980
                                         ...  
snowthane                                    1
metalaxyl, matco, systemil, mancozebe        1
metalaxyl, mancozan, mancozeb                1
hexaconazole, silicon, mancozeb              1
cymoxanil, aqua, mancozeb                    1
Name: Product, Length: 147, dtype: int64

In [25]:
df2.columns

Index(['Type', 'Year', 'Month', 'Date', 'IMPORTER/EXPORTER NAME',
       'ORIGIN/DESTINATION COUNTRY', 'PRODUCT DESCRIPTION', 'QTY', 'UNIT',
       'UNIT PRICE (USD)', 'Total_VALUE_USD', 'INVOICE CURRENCY',
       'SUPPLIER/BUYER NAME', 'Country', 'Company', 'Product', 'suffix'],
      dtype='object')

### ----------------------------------------------------------- SUFFIX -------------------------------------------------------------------------

In [26]:
def suf(x):
    temp_suffix = [] 
    if re.search('tech.{,50}|TECH.{,50}',x):
        temp_suffix = "TECH"
    elif re.search('IPA SALT|ipa salt',x):
        temp_suffix = "IPA SALT"
    elif re.search('62% IPA SALT|62% ipa salt',x):
        temp_suffix = "62% IPA SALT"
    elif re.search('41% SL|41% sl',x):
        temp_suffix = '41% SL'
    elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
        temp_suffix = "ALION PLUS"
    else :
        temp_suffix = "Others"
    return temp_suffix

In [27]:
df2['suffix'] =  df2['PRODUCT DESCRIPTION'].apply(lambda x: suf(x))

In [28]:
df2.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Country,Company,Product,suffix
0,Import,2021,1,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,NaN,KRISHI RASAYAN,emamectin,TECH
1,Import,2021,1,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,NaN,WILLOWOOD CHEMICALS PRIVATE LIMITED,emamectin,TECH
2,Import,2021,2,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,NaN,SYNGENTA INDIA LIMITED,"lufenuron, emamectin",Others
3,Import,2021,2,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,NaN,SYNGENTA INDIA LIMITED,"lufenuron, emamectin",Others
4,Import,2021,3,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,NaN,CRYSTAL CROP PROTECTION LIMITED,emamectin,TECH


_______________________________________________________________

_______________________________________________________________

## Working on the data from 2022

In [29]:
d1 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\Jan to July\38_Exp_TY2_Jan_to_Jul-22.csv")
d2 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\Jan to July\38_Imp_TY2_Jan_to_Jul-22.csv")
d3 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\August\38_Exp_TY2_Aug-22.csv")
d4 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\August\38_Imp_TY2_Aug-22.csv")
d5 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\September\38_Exp_TY2_Sep-22.csv")
d6 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\September\38_Imp_TY2_Sep-22.csv")
d7 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\October\38_Exp_TY2_Oct-22.csv")
d8 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\October\38_Imp_TY2_Oct-22.csv")
d9 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\November\38_Exp_TY2_Nov-22.csv")
d10 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\November\38_Imp_TY2_Nov-22.csv")
d11 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\December\38_Exp_TY2_Dec-22.csv")
d12 = pd.read_csv(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Updated File\December\38_Imp_TY2_Dec-22.csv")

In [30]:
list1 = list[d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12]

In [31]:
print(d1.shape)
print(d2.shape)
print(d3.shape)
print(d4.shape)
print(d5.shape)
print(d6.shape)
print(d7.shape)
print(d8.shape)
print(d9.shape)
print(d10.shape)
print(d11.shape)
print(d12.shape)

(164259, 25)
(395195, 28)
(26131, 25)
(56233, 28)
(27751, 25)
(56301, 28)
(23914, 25)
(51918, 28)
(28451, 25)
(53933, 28)
(31409, 25)
(56360, 28)


In [32]:
d_exp = pd.concat([d1,d3,d5,d7,d9,d11])

In [33]:
d_exp.shape

(301915, 25)

In [34]:
d_imp =  pd.concat([d2,d4,d6,d8,d10,d12])

In [35]:
d_imp.shape

(669940, 29)

## Dealing with the Import File

In [36]:
### Block for altering the entire data into the required format


temp = d_imp.drop(['Port Code', 'Indian Port', 'Declaration No','Address', 'City', 'CHA No', 'CHA Name','HS Code','Total_Duty_Usd', 'Loading Port','Supplier Address', 'BE Type', 'IEC Phone','IEC Email', 'IEC Contact'],axis = 1)
temp['Product Name'] = temp['Product'] + " " + temp['suffix']
temp = temp.drop('suffix',axis = 1)
temp['Type'] = 'Import'
temp['Year'] = pd.DatetimeIndex(temp['Date']).year
temp['Month'] = pd.DatetimeIndex(temp['Date']).month
temp.rename(columns = {'Importer Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Price Usd':'UNIT PRICE (USD)','Total Value Usd':'Total_VALUE_USD','Inv Currency':'INVOICE CURRENCY','Supplier Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)
temp['ORIGIN/DESTINATION COUNTRY'] = temp['Country']
temp['Company'] = temp['IMPORTER/EXPORTER NAME']


temp = temp.loc[:,["Type","Year","Month","Date","IMPORTER/EXPORTER NAME","ORIGIN/DESTINATION COUNTRY","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Country","Product Name","Company"]]
temp['Product Name'].replace('Others Others','Others',inplace = True)

In [37]:
temp.shape

(669940, 17)

In [38]:
temp.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Country,Product Name,Company
0,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1396342_METFORMIN RELATED COMPOUND C -(25 ...,4.0,NOS,734.136941,2936.547764,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
1,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1432005_METHYLPARABEN -(125 MG)-(PHARMACEU...,19.0,NOS,198.046246,3762.878678,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
2,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1441298_METOPROLOL SUCCINATE -(200 MG)-(PH...,4.0,NOS,421.701912,1686.807647,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
3,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1446815_MONTELUKAST DICYCLOHEXYLAMINE -(15...,6.0,NOS,740.112486,4440.674919,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
4,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1448617_MOXIFLOXACIN RELATED COMPOUND A -(...,3.0,NOS,734.136963,2202.410888,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED


## Dealing with the Import File

In [39]:
### Block for altering the entire data into the required format

temp2 = d_exp.drop(['Declaration No', 'HS Four Digit', 'HS Code','Destination Port','Indian Port','Address', 'City','Consinee Address','Port Code', 'IEC Phone', 'IEC Email', 'IEC Contact'],axis = 1)
temp2['Product Name'] = temp2['Product'] + " " + temp2['suffix']
temp2 = temp2.drop('suffix',axis = 1)
temp2['Type'] = 'Export'
temp2['Year'] = pd.DatetimeIndex(temp2['Date']).year
temp2['Month'] = pd.DatetimeIndex(temp2['Date']).month
temp2.rename(columns = {'Exporter Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Value Usd':'UNIT PRICE (USD)','Fob Usd':'Total_VALUE_USD','Currency':'INVOICE CURRENCY','Consinee Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)
temp2['ORIGIN/DESTINATION COUNTRY'] = temp2['Country']
temp2['Company'] = temp2['IMPORTER/EXPORTER NAME']

temp2 = temp2.loc[:,["Type","Year","Month","Date","IMPORTER/EXPORTER NAME","ORIGIN/DESTINATION COUNTRY","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Country","Product Name","Company"]]
temp2['Product Name'].replace('Others Others','Others',inplace = True)

In [40]:
temp2.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Country,Product Name,Company
0,Export,2022,1,2022-01-01,UPL LIMITED,UNITED STATES,MANZATE MAX FORMULATION (FUNGICIDE),17929.62,KGS,1.814029,32524.850895,USD,UPL NA INC,manzate,UNITED STATES,manzate Others,UPL LIMITED
1,Export,2022,1,2022-01-01,MEGHMANI ORGANICS LIMITED,UNITED STATES,"INDIAN ORIGIN 2,4 DI-CHLORO PHENOXY ACET",90000.00,KGS,3.853177,346785.950961,USD,TO THE ORDER OF G,Others,UNITED STATES,Others,MEGHMANI ORGANICS LIMITED
2,Export,2022,1,2022-01-01,KAKADIYA CHEMICAL,SRI LANKA,"DISINFECTANT, SOLID, TOXIC, N.O.S. AVATO",53.10,MTS,435.553659,23127.899271,USD,TO THE ORDER OF G,avato,SRI LANKA,avato Others,KAKADIYA CHEMICAL
3,Export,2022,1,2022-01-01,KAKADIYA CHEMICAL,SRI LANKA,"DISINFECTANT, SOLID, TOXIC, N.O.S. AVATO",53.10,MTS,435.553659,23127.899271,USD,TO THE ORDER OF G,avato,SRI LANKA,avato Others,KAKADIYA CHEMICAL
4,Export,2022,1,2022-01-01,PERSANG ALLOY INDUSTRIES PRIVATE LIMITED,SAUDI ARABIA,SOLER PASTE FOR SOLDERING SH 6309 TYPE 4,20.00,KGS,140.490391,2809.807820,USD,PIONEER TECHNICAL SYSTE,Others,SAUDI ARABIA,Others,PERSANG ALLOY INDUSTRIES PRIVATE LIMITED


In [41]:
temp2.shape

(301915, 17)

In [42]:
data_2022 = pd.concat([temp,temp2])

In [43]:
data_2022.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Country,Product Name,Company
0,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1396342_METFORMIN RELATED COMPOUND C -(25 ...,4.0,NOS,734.136941,2936.547764,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
1,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1432005_METHYLPARABEN -(125 MG)-(PHARMACEU...,19.0,NOS,198.046246,3762.878678,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
2,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1441298_METOPROLOL SUCCINATE -(200 MG)-(PH...,4.0,NOS,421.701912,1686.807647,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
3,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1446815_MONTELUKAST DICYCLOHEXYLAMINE -(15...,6.0,NOS,740.112486,4440.674919,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED
4,Import,2022,1,2022-01-01,CHROMACHEMIE LABORATORY PRIVATE LIMITED,UNITED STATES,USP-1448617_MOXIFLOXACIN RELATED COMPOUND A -(...,3.0,NOS,734.136963,2202.410888,USD,THE UNITED STATES PHARMACOPEIA,Others,UNITED STATES,Others,CHROMACHEMIE LABORATORY PRIVATE LIMITED


In [44]:
data_2022.isnull().sum()/len(data_2022)

Type                          0.000000
Year                          0.000000
Month                         0.000000
Date                          0.000000
IMPORTER/EXPORTER NAME        0.000002
ORIGIN/DESTINATION COUNTRY    0.000010
PRODUCT DESCRIPTION           0.000001
QTY                           0.000000
UNIT                          0.000000
UNIT PRICE (USD)              0.000000
Total_VALUE_USD               0.000000
INVOICE CURRENCY              0.000000
SUPPLIER/BUYER NAME           0.004542
Molecule Name                 0.000000
Country                       0.000010
Product Name                  0.000000
Company                       0.000002
dtype: float64

In [45]:
df2.head()

,Type,Year,Month,Date,IMPORTER/EXPORTER NAME,ORIGIN/DESTINATION COUNTRY,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Country,Company,Product,suffix
0,Import,2021,1,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,NaN,KRISHI RASAYAN,emamectin,TECH
1,Import,2021,1,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,NaN,WILLOWOOD CHEMICALS PRIVATE LIMITED,emamectin,TECH
2,Import,2021,2,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,NaN,SYNGENTA INDIA LIMITED,"lufenuron, emamectin",Others
3,Import,2021,2,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,NaN,SYNGENTA INDIA LIMITED,"lufenuron, emamectin",Others
4,Import,2021,3,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,NaN,CRYSTAL CROP PROTECTION LIMITED,emamectin,TECH


In [46]:
df2.columns

Index(['Type', 'Year', 'Month', 'Date', 'IMPORTER/EXPORTER NAME',
       'ORIGIN/DESTINATION COUNTRY', 'PRODUCT DESCRIPTION', 'QTY', 'UNIT',
       'UNIT PRICE (USD)', 'Total_VALUE_USD', 'INVOICE CURRENCY',
       'SUPPLIER/BUYER NAME', 'Country', 'Company', 'Product', 'suffix'],
      dtype='object')

In [47]:
data_2022.columns

Index(['Type', 'Year', 'Month', 'Date', 'IMPORTER/EXPORTER NAME',
       'ORIGIN/DESTINATION COUNTRY', 'PRODUCT DESCRIPTION', 'QTY', 'UNIT',
       'UNIT PRICE (USD)', 'Total_VALUE_USD', 'INVOICE CURRENCY',
       'SUPPLIER/BUYER NAME', 'Molecule Name', 'Country', 'Product Name',
       'Company'],
      dtype='object')

In [48]:
### Block for altering the entire data into the required format
temp3 = df2
temp3['Product Name'] = temp3['Product'] + " " + temp3['suffix']
temp3 = temp3.drop('suffix',axis = 1)
temp3.rename(columns = {'Product':'Molecule Name'},inplace = True)


temp3 = temp3.loc[:,["Type","Year","Month","Date","IMPORTER/EXPORTER NAME","ORIGIN/DESTINATION COUNTRY","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Country","Product Name","Company"]]
temp3['Product Name'].replace('Others Others','Others',inplace = True)

In [49]:
temp3.columns

Index(['Type', 'Year', 'Month', 'Date', 'IMPORTER/EXPORTER NAME',
       'ORIGIN/DESTINATION COUNTRY', 'PRODUCT DESCRIPTION', 'QTY', 'UNIT',
       'UNIT PRICE (USD)', 'Total_VALUE_USD', 'INVOICE CURRENCY',
       'SUPPLIER/BUYER NAME', 'Molecule Name', 'Country', 'Product Name',
       'Company'],
      dtype='object')

In [50]:
data_2022.columns

Index(['Type', 'Year', 'Month', 'Date', 'IMPORTER/EXPORTER NAME',
       'ORIGIN/DESTINATION COUNTRY', 'PRODUCT DESCRIPTION', 'QTY', 'UNIT',
       'UNIT PRICE (USD)', 'Total_VALUE_USD', 'INVOICE CURRENCY',
       'SUPPLIER/BUYER NAME', 'Molecule Name', 'Country', 'Product Name',
       'Company'],
      dtype='object')

In [51]:
final_data = pd.concat([temp3,data_2022])

In [52]:
final_data.shape

(996450, 17)

In [53]:
final_data.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\December.xlsx",sheet_name = 'Data',index = False)

In [94]:
final_data["Product Name"].value_counts()

Others                           839191
oil Others                        13580
mancozeb Others                   13328
calcium Others                     7588
control Others                     6530
                                  ...  
diamond, oil Others                   1
thiabendazole, biocide Others         1
potassium, zinc Others                1
total, aqua Others                    1
ready, urea Others                    1
Name: Product Name, Length: 1943, dtype: int64

In [54]:
final_data.isnull().sum()

Type                              0
Year                              0
Month                             0
Date                              0
IMPORTER/EXPORTER NAME            2
ORIGIN/DESTINATION COUNTRY       10
PRODUCT DESCRIPTION               1
QTY                               0
UNIT                              0
UNIT PRICE (USD)                  0
Total_VALUE_USD                   0
INVOICE CURRENCY                  1
SUPPLIER/BUYER NAME            5041
Molecule Name                     0
Country                       24605
Product Name                      0
Company                           2
dtype: int64

In [ ]:
final_data.loc[final_data['IMPORTER/EXPORTER NAME']]